In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [10]:
url = 'http://127.0.0.1:25510/list/roots?sec=STOCK'
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # The request was successful
    json_data = response.json()
    print(json_data)
else:
    # Request failed
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

{'header': {'id': 19, 'latency_ms': 164, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': ['A', 'AA', 'AAA', 'AAAP', 'AAAU', 'AABA', 'AAC', 'AAC.U', 'AAC.WS', 'AACC', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACOU', 'AACOW', 'AACQ', 'AACQU', 'AACQW', 'AACT', 'AACT.U', 'AACT.WS', 'AADI', 'AADR', 'AAGR', 'AAGRW', 'AAIC', 'AAIC.PR.B', 'AAIC.PR.C', 'AAIN', 'AAIR', 'AAIT', 'AAL', 'AALCP', 'AAM.PR.A', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPC', 'AAPD', 'AAPH', 'AAPL', 'AAPU', 'AAPX', 'AAPY', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAVL', 'AAWW', 'AAXJ', 'AAXN', 'AB', 'ABAC', 'ABAT', 'ABAX', 'ABB', 'ABBB', 'ABBV', 'ABC', 'ABCB', 'ABCD', 'ABCL', 'ABCM', 'ABCO', 'ABCS', 'ABCW', 'ABDC', 'ABEO', 'ABEOW', 'ABEQ', 'ABEV', 'ABFS', 'ABG', 'ABGB', 'ABGI', 'ABHD', 'ABIL', 'ABILW', 'ABIO', 'ABL', 'ABLLL', 'ABLLW', 'ABLV', 'ABLVW', 'ABLX', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR.PR.E', 'ABSI', 'ABST', 'ABT', 'ABTL', 'ABTS', 'ABTX', 'A

In [3]:
from enum import Enum

class Security(Enum):
    OPTION = 'option'
    EQUITY = 'stock'
    
class Req(Enum):
    TRADE = 'trade'
    QUOTE = 'quote'
    OI = 'open_interest'

class Right(Enum):
    CALL = 'C'
    PUT = 'P'

In [12]:
class ThetaDataAPI:
    def __init__(self):
        self.base_url = 'http://127.0.0.1:25510/'

    def _get_req(self, url: str, headers: dict, params: dict=None):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

    def get_roots(self, security_type: Security=Security.OPTION):
        url = f'{self.base_url}list/roots?sec={security_type.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_expirations(self, root:str):
        url = f'{self.base_url}list/expirations?root={root}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_strikes(self, root: str, exp: str):
        url = f'{self.base_url}list/strikes?root={root}&exp={exp}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_dates(self, root: str, security_type: Security=Security.OPTION, exp: str=None, strike_right: tuple[str, Right]=None):
        url = f'{self.base_url}list/dates/{security_type.value}/quote'
        url_params = {}
        url_params["root"] = root
        if exp is not None:
            url_params["exp"] = exp
        if strike_right is not None:
            url_params["strike"] = strike_right[0]
            url_params["right"] = strike_right[1].value
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers, params=url_params)
    
    """
    NOTE this function has a 750ms overhead 
    """
    def get_contracts(self, date: str, req: Req=Req.TRADE):
        url = f'{self.base_url}list/contracts/option/{req.value}?start_date={date}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
        
    def get_eod_prices(self, root: str, security_type: Security, start_date: str, end_date: str, exp: str=None, strike: str=None, right: Right=None):
        url = None
        if security_type == Security.EQUITY:
            url = f'{self.base_url}hist/{security_type.value}/eod?root={root}&start_date={start_date}&end_date={end_date}'
        elif security_type == Security.OPTION:
            url = f'{self.base_url}hist/{security_type.value}/eod?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        print(url)
        return self._get_req(url=url, headers=headers)

    def get_hist_quotes(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str=None):
        url = f'{self.base_url}hist/option/quote?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={0 if ivl is None else ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_ohlc(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/ohlc?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_oi(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/open_interest?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_trades(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right):
        url = f'{self.base_url}hist/option/trade?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_hist_trade_quotes(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right):
        url = f'{self.base_url}hist/option/trade_quote?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_iv(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/implied_volatility?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_hist_iv_verbose(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str):
        url = f'{self.base_url}hist/option/implied_volatility_verbose?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_eod_greeks(self, root: str, exp: str, start_date: str, end_date: str):
        url = f'{self.base_url}bulk_hist/option/eod_trade_greeks?root={root}&exp={exp}&start_date={start_date}&end_date={end_date}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)


In [14]:
client = ThetaDataAPI()
root = "AAPL"

In [6]:
cached_eod_greeks = client.get_eod_greeks(root=root, exp=20230406, start_date=20230329, end_date=20230331)

In [105]:
class WrapperClient:
    def __init__(self):
        self.thetadata = ThetaDataAPI()
        self.date_format = "%Y%m%d"

    def get_dates_in_range(self, root: str, exp: str, start_date: str, end_date: str):
        dates = self.thetadata.get_dates(root=root, exp=exp)["response"]
        start = start_date if start_date is not None else dates[0]
        end = end_date if end_date is not None else dates[len(dates)-1]
        start_datetime = dt.strptime(str(start), self.date_format)
        end_datetime = dt.strptime(str(end), self.date_format)
        dates = [date for date in dates if start_datetime <= dt.strptime(str(date), self.date_format) <= end_datetime]
        return (start, end, dates)
    
    # return eod data about an underlying over some range of time
    def get_underlying_over_time(self, root: str, security_type: Security, points: list[str], dates: list[str]):
        num_days = len(dates)
        data = {}
        for point in points:
            data[point] = [None] * num_days
        data_points = {}
        for i in range(num_days):
            eod_prices = self.thetadata.get_eod_prices(root=root, security_type=security_type, start_date=dates[i], end_date=dates[i])
            header = eod_prices["header"]
            response = eod_prices["response"]
            # initialize data_points based on format
            if not data_points:
                for point in points:
                    data_points[point] = header["format"].index(point)
            for point, index in data_points.items():
                data[point][i] = response[0][index]
        return data

    # return interval NBBO of a chain over some range of time
    def get_chain_over_time(self, root: str, exp: str, strike: str, right: Right, ivl: int, start_date: str, end_date: str):
        ret = []
        data = self.thetadata.get_hist_quotes(root=root, start_date=start_date, end_date=end_date, exp=exp, strike=strike, right=right, ivl=3600000)
        format = data["header"]["format"]
        response = data["response"]
        ask_ind = format.index("ask")
        bid_ind = format.index("bid")
        date_ind = format.index("date")
        timestamp_ind = format.index("ms_of_day")
        for row in response:
            ret.append([row[date_ind], row[timestamp_ind], row[bid_ind], row[ask_ind]])
        return pd.DataFrame(ret, columns=["date", "ms_of_day", "bid", "ask"])

    # garbage function for testing
    def test_get_chain_over_time(self, root: str, exp: str, strike: str, right: Right, dates: list[str]):
        prices = []
        for date in dates:
            data = self.thetadata.get_hist_trades(root=root, start_date=date, end_date=date, exp=exp, strike=strike, right=right)
            header = data["header"]
            if header["error_type"] != "null":
                print(strike, date)
                #print(data)
                continue
            ind = header["format"].index("price")
            prices.append(data["response"][-1][ind])
        print(prices)
        

    def get_chains_over_time(self, root: str, exp: str, right: Right, points: list[str], start_date: str=None, end_date: str=None):
        start, end, dates = self.get_dates_in_range(root=root, exp=exp, start_date=start_date, end_date=end_date)
        num_days = len(dates)
        strikes = self.thetadata.get_strikes(root=root, exp=exp)["response"]
        num_strikes = len(strikes)
        data = {}
        for point in points:
            data[point] = []
        data_points = {}
        valid_strikes = []
        for i in range(num_strikes):
            eod_prices = self.thetadata.get_hist_quotes(root=root, start_date=start, end_date=end, exp=exp, strike=strikes[i], right=right)
            header = eod_prices["header"]
            # skip if there was error with data
            if header["error_type"] != "null":
                continue
            response = eod_prices["response"]
            if not data_points:
                for point in points:
                    data_points[point] = header["format"].index(point)
            for point, index in data_points.items():
                data[point].append([d[index] for d in response])
        for point in points:
            data[point] = np.column_stack(data[point])
        print(data)
        return (dates, valid_strikes, data)


    # Need to address blanks in data
    def get_greeks_chains_over_time(self, root: str, exp: str, points: list[str], start_date: str=None, end_date: str=None):
        start, end, dates = self.get_dates_in_range(root=root, exp=exp, start_date=start_date, end_date=end_date)
        num_days = len(dates)
        eod_greeks = cached_eod_greeks #self.thetadata.get_eod_greeks(root=root, start_date=start, end_date=end, exp=exp)
        header = eod_greeks["header"]
        response = eod_greeks["response"]
        data_points = {}
        calls = {}
        puts = {}
        for point in points:
            data_points[point] = header["format"].index(point)
            calls[point] = {}
            puts[point] = {}
        for eod in response:
            contract = eod["contract"]
            ticks = eod["ticks"]
            for point, index in data_points.items():
                (calls if contract["right"] == "C" else puts)[point][contract["strike"]] = [d[index] for d in ticks]
        for point in points:
            print(list(calls[point].values()))
            # calls[point] = {k: v for k, v in calls[point].items() if len(v)==num_days}
            # calls[point] = (calls[point].keys(), np.column_stack(list(calls[point].values())))
        return (calls, puts)



In [106]:
wrapper_client = WrapperClient()

In [96]:
dates = wrapper_client.get_dates_in_range(root="AAPL", exp=20220930, start_date=20220910, end_date=20220928)[2]
strikes = [135000, 140000, 145000] #client.get_strikes(root="AAPL", exp=20220930)["response"]
for K in strikes:
    print(wrapper_client.get_chain_over_time(root="AAPL", exp=20220930, strike=K, right=Right.CALL, dates=dates))

[28.7, 19.5, 20.85, 17.95, 15.55, 19.9, 22.15, 19.0, 17.9, 15.45, 15.9, 16.05, 14.75] [28.85, 21.6, 21.05, 18.1, 16.5, 20.0, 22.45, 19.25, 18.1, 16.05, 16.05, 17.7, 15.05]
[28.775 20.55  20.95  18.025 16.025 19.95  22.3   19.125 18.    15.75
 15.975 16.875 14.9  ]
None
[23.8, 14.75, 16.15, 13.4, 11.25, 15.15, 17.35, 14.3, 13.1, 10.7, 11.15, 11.85, 9.95] [24.0, 16.95, 16.35, 13.5, 12.15, 15.3, 17.65, 14.5, 13.3, 11.35, 13.6, 12.75, 10.2]
[23.9   15.85  16.25  13.45  11.7   15.225 17.5   14.4   13.2   11.025
 12.375 12.3   10.075]
None
[19.0, 10.65, 11.75, 9.25, 7.0, 10.7, 12.8, 9.85, 8.65, 6.9, 6.85, 7.05, 5.35] [19.2, 10.8, 11.9, 9.35, 8.0, 10.8, 13.0, 10.05, 8.85, 7.05, 7.0, 7.75, 5.5]
[19.1   10.725 11.825  9.3    7.5   10.75  12.9    9.95   8.75   6.975
  6.925  7.4    5.425]
None


In [108]:
wrapper_client.get_chain_over_time(root="SVXY", exp=20230317, strike=55000, right=Right.CALL, ivl=3600000, start_date=20230117, end_date=20230317)

,date,ms_of_day,bid,ask
0,20230117,36000000,9.00,9.70
1,20230117,39600000,8.50,9.50
2,20230117,43200000,9.20,9.80
3,20230117,46800000,9.40,10.20
4,20230117,50400000,9.50,9.90
...,...,...,...,...
296,20230317,43200000,3.10,3.40
297,20230317,46800000,3.10,3.30
298,20230317,50400000,3.10,3.50
299,20230317,54000000,2.60,2.85


In [93]:
client.get_hist_quotes(root="SVXY", start_date=20230301, end_date=20230315, exp=20230317, strike=60000, right=Right.CALL, ivl=3600000)

{'header': {'id': 2373,
  'latency_ms': 91,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'bid_size',
   'bid_condition',
   'bid',
   'bid_exchange',
   'ask_size',
   'ask_condition',
   'ask',
   'ask_exchange',
   'date']},
 'response': [[36000000, 36, 11, 4.3, 50, 10, 9, 5.9, 50, 20230301],
  [39600000, 43, 9, 4.8, 50, 495, 5, 5.2, 50, 20230301],
  [43200000, 30, 9, 4.8, 50, 414, 5, 5.1, 50, 20230301],
  [46800000, 30, 9, 4.8, 50, 351, 5, 5.1, 50, 20230301],
  [50400000, 30, 9, 4.7, 50, 423, 5, 5.0, 50, 20230301],
  [54000000, 1, 65, 4.6, 50, 47, 9, 4.8, 50, 20230301],
  [57600000, 490, 9, 4.5, 50, 289, 9, 5.0, 50, 20230301],
  [36000000, 448, 9, 4.6, 50, 361, 9, 5.0, 50, 20230302],
  [39600000, 2, 65, 5.0, 50, 1, 65, 5.2, 50, 20230302],
  [43200000, 6, 4, 5.1, 50, 1, 4, 5.3, 50, 20230302],
  [46800000, 4, 4, 5.1, 50, 4, 4, 5.3, 50, 20230302],
  [50400000, 4, 4, 5.4, 50, 40, 65, 5.7, 50, 20230302],
  [54000000, 2, 4, 5.6, 50, 17

In [ ]:
65000 20230126
65000 20230131

In [60]:
print(client.get_expirations(root="SVIX"))

{'header': {'id': 914, 'latency_ms': 35, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': [20220414, 20220520, 20220617, 20220715, 20220819, 20220916, 20221021, 20221118, 20221216, 20230120, 20230217, 20230317, 20230421, 20230519, 20230616, 20230721, 20230818, 20230915, 20231020, 20231117, 20231201, 20231208, 20231215, 20231222, 20231229, 20240105, 20240112, 20240119, 20240126, 20240202, 20240209, 20240216, 20240223, 20240301, 20240315, 20240419, 20240517, 20240621, 20240920, 20250117, 20260116]}


In [11]:
expiries = client.get_expirations(root=root)["response"]
exp = expiries[len(expiries)-50]
strikes = client.get_strikes(root=root, exp=exp)["response"]
strikes = strikes[30:-20]
dates = client.get_dates(root=root, exp=exp)["response"]
print(exp, dates[20])
for K in strikes:
    print(K, ":", client.get_eod_prices(root=root, security_type=Security.OPTION, start_date=dates[20], end_date=dates[20], exp=exp, strike=K, right=Right.CALL))

20230630 20230609
http://127.0.0.1:25510/hist/option/eod?root=AAPL&start_date=20230609&end_date=20230609&strike=177500&exp=20230630&right=C
177500 : {'header': {'id': 25, 'latency_ms': 35, 'error_type': 'NO_DATA', 'error_msg': 'No data for the specified timeframe & contract.', 'next_page': 'null', 'format': None}, 'response': [0]}
http://127.0.0.1:25510/hist/option/eod?root=AAPL&start_date=20230609&end_date=20230609&strike=180000&exp=20230630&right=C
180000 : {'header': {'id': 26, 'latency_ms': 34, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': ['open', 'high', 'low', 'close', 'volume', 'count', 'date']}, 'response': [[4.4, 4.9, 3.85, 4.0, 1950, 390, 20230609]]}
http://127.0.0.1:25510/hist/option/eod?root=AAPL&start_date=20230609&end_date=20230609&strike=182500&exp=20230630&right=C
182500 : {'header': {'id': 27, 'latency_ms': 34, 'error_type': 'NO_DATA', 'error_msg': 'No data for the specified timeframe & contract.', 'next_page': 'null', 'format': None}, 'res

In [15]:
contracts = client.get_contracts(date=20220512)
print(contracts["header"])
print(contracts["response"][:10])

{'id': 10, 'latency_ms': 2497, 'error_type': 'null', 'error_msg': 'null', 'next_page': None, 'format': ['root', 'expiration', 'strike', 'right']}
[['BLK', 20220513, 670000, 'C'], ['ARKK', 20230120, 47180, 'C'], ['BLK', 20220610, 670000, 'C'], ['AVGO', 20220624, 670000, 'C'], ['BX', 20220513, 113000, 'P'], ['AMD', 20220513, 113000, 'C'], ['COP', 20220513, 113000, 'C'], ['BABA', 20220513, 113000, 'C'], ['AAPL', 20240621, 145000, 'C'], ['AAPL', 20240621, 145000, 'P']]


In [131]:
eod = client.get_eod_prices(root=root, start_date=20230301, end_date=20230406, exp=20230406, strike=150000, right=Right.CALL)
eod

{'header': {'id': 2,
  'latency_ms': 49,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['open', 'high', 'low', 'close', 'volume', 'count', 'date']},
 'response': [[3.95, 3.95, 3.13, 3.2, 319, 55, 20230301],
  [2.8, 3.5, 2.59, 3.3, 484, 67, 20230302],
  [4.05, 5.6, 3.75, 5.6, 1079, 217, 20230303],
  [7.33, 9.05, 7.0, 7.3, 752, 135, 20230306],
  [7.22, 7.35, 5.6, 5.9, 294, 65, 20230307],
  [6.6, 6.95, 6.2, 6.5, 274, 53, 20230308],
  [6.8, 7.55, 5.35, 5.4, 197, 52, 20230309],
  [5.05, 5.4, 4.01, 4.26, 430, 135, 20230310],
  [4.4, 6.7, 4.16, 5.3, 463, 136, 20230313],
  [5.86, 6.5, 4.88, 5.76, 1547, 160, 20230314],
  [5.55, 6.7, 4.98, 6.32, 301, 73, 20230315],
  [6.2, 8.5, 6.0, 8.15, 176, 54, 20230316],
  [7.75, 8.71, 7.4, 7.7, 79, 37, 20230317],
  [7.7, 9.45, 7.55, 9.23, 62, 40, 20230320],
  [9.71, 10.48, 8.56, 10.43, 88, 36, 20230321],
  [10.85, 12.44, 9.73, 9.73, 145, 58, 20230322],
  [10.3, 12.1, 9.56, 10.37, 83, 37, 20230323],
  [10.07, 10.9, 9.4, 10

In [ ]:
quotes = client.get_hist_quotes(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=60000)
quotes

In [ ]:
ohlc = client.get_ohlc(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=900000)
ohlc

In [ ]:
oi = client.get_hist_oi(root=root, start_date=20230301, end_date=20230401, exp=20230406, strike=150000, right=Right.CALL, ivl=900000)
oi

In [14]:
trades = client.get_hist_trades(root=root, start_date=20230301, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL)
trades

{'header': {'id': 9,
  'latency_ms': 395,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day', 'sequence', 'size', 'condition', 'price', 'date']},
 'response': [[34200221, 1797808825, 1, 18, 3.95, 20230301],
  [34523772, 1818045057, 2, 18, 3.8, 20230301],
  [35338442, 1862555299, 1, 18, 3.9, 20230301],
  [36111207, 1903553439, 1, -125, 3.94, 20230301],
  [37063665, 1953538306, 1, 18, 3.45, 20230301],
  [37210498, 1959519491, 20, 18, 3.5, 20230301],
  [37665891, 1977700536, 4, -125, 3.4, 20230301],
  [38136560, 1996538861, 20, 18, 3.55, 20230301],
  [38709722, 2020434456, 1, -126, 3.79, 20230301],
  [38751597, 2021796160, 2, -126, 3.76, 20230301],
  [39740240, 2056265330, 1, 125, 3.6, 20230301],
  [39849996, 2059810248, 18, -125, 3.54, 20230301],
  [39849996, 2059810250, 18, -125, 3.54, 20230301],
  [39910651, 2061879890, 1, 18, 3.55, 20230301],
  [40067808, 2066919131, 1, 125, 3.55, 20230301],
  [40524877, 2085890189, 1, -126, 3.35, 20230301],

In [17]:
quotes = client.get_hist_quotes(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL)
quotes

{'header': {'id': 12,
  'latency_ms': 117,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'bid_size',
   'bid_condition',
   'bid',
   'bid_exchange',
   'ask_size',
   'ask_condition',
   'ask',
   'ask_exchange',
   'date']},
 'response': [[28800211, 0, 60, 0.0, 50, 0, 60, 0.0, 50, 20230331],
  [28800257, 0, 42, 0.0, 50, 0, 42, 0.0, 50, 20230331],
  [28800326, 0, 65, 0.0, 50, 0, 65, 0.0, 50, 20230331],
  [28801788, 0, 5, 0.0, 50, 0, 5, 0.0, 50, 20230331],
  [30600163, 0, 65, 0.0, 50, 0, 65, 0.0, 50, 20230331],
  [30600195, 0, 42, 0.0, 50, 0, 42, 0.0, 50, 20230331],
  [30600196, 0, 60, 0.0, 50, 0, 60, 0.0, 50, 20230331],
  [30601668, 0, 5, 0.0, 50, 0, 5, 0.0, 50, 20230331],
  [32400090, 0, 60, 0.0, 50, 0, 60, 0.0, 50, 20230331],
  [32400126, 0, 42, 0.0, 50, 0, 42, 0.0, 50, 20230331],
  [32400253, 0, 5, 0.0, 50, 0, 5, 0.0, 50, 20230331],
  [32400340, 0, 65, 0.0, 50, 0, 65, 0.0, 50, 20230331],
  [33900204, 0, 31, 0.0, 50, 0, 31, 0.0, 5

In [13]:
iv = client.get_hist_iv(root=root, start_date=20230301, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL, ivl=3600000)
iv

{'header': {'id': 8,
  'latency_ms': 14522,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'bid',
   'bid_implied_vol',
   'midpoint',
   'implied_vol',
   'ask',
   'ask_implied_vol',
   'underlying_price',
   'date']},
 'response': [[36000000,
   3.5,
   0.2478,
   3.9,
   0.2701,
   4.3,
   0.292,
   146.92,
   20230301],
  [39600000, 3.65, 0.2671, 3.7, 0.2699, 3.75, 0.2725, 146.47, 20230301],
  [43200000, 3.65, 0.2672, 3.67, 0.2687, 3.7, 0.2703, 146.45, 20230301],
  [46800000, 3.6, 0.2664, 3.62, 0.2674, 3.65, 0.2691, 146.38, 20230301],
  [50400000, 3.4, 0.267, 3.42, 0.2682, 3.45, 0.27, 145.87, 20230301],
  [54000000, 3.25, 0.2677, 3.27, 0.2693, 3.3, 0.2702, 145.48, 20230301],
  [36000000, 2.74, 0.2669, 2.76, 0.2681, 2.78, 0.2693, 144.34, 20230302],
  [39600000, 2.75, 0.2643, 2.76, 0.2649, 2.78, 0.2661, 144.48, 20230302],
  [43200000, 2.69, 0.2635, 2.71, 0.2647, 2.73, 0.2659, 144.35, 20230302],
  [46800000, 2.71, 0.2626, 2.73, 0.26

In [69]:
eod_greeks = client.get_eod_greeks(root=root, exp=20230406, start_date=20230329, end_date=20230331)
eod_greeks

{'header': {'id': 365,
  'latency_ms': 80464,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': ['ms_of_day',
   'delta',
   'theta',
   'vega',
   'rho',
   'epsilon',
   'lambda',
   'gamma',
   'vanna',
   'charm',
   'vomma',
   'veta',
   'implied_vol',
   'open',
   'high',
   'low',
   'close',
   'volume',
   'count',
   'sequence',
   'condition',
   'size',
   'exchange',
   'price',
   'condition_flags',
   'price_flags',
   'volume_type',
   'records_back',
   'underlying_price',
   'date']},
 'response': [{'ticks': [[39256716,
     0.9999,
     -0.0069,
     0.0029,
     1.2312,
     -3.93,
     1.4562,
     0.0,
     -0.0002,
     0.0087,
     0.023,
     0.0005,
     1.9018,
     0.0,
     0.0,
     0.0,
     0.0,
     0,
     0,
     1474929125,
     18,
     1,
     0,
     109.45,
     0,
     0,
     0,
     0,
     159.39,
     20230329],
    [39256716,
     0.9998,
     -0.009,
     0.017,
     1.094,
     -3.5557,
     1.4444,
     